# Customer Review Processing Pipeline with Firehose Data Streams

1. Reviews are submitted to Firehose Streams
2. Firehose initiates data transformation through Lambda.
3. Lambda invokes Comprehend to assess sentiment and appends sentiment to JSON
4. Firehose gathers the transformed data and stores it in S3.
5. This pipeline enables Firehose to continuously receive, process, and transfer streaming data to S3

Input: Customer Review
Output: Overall sentiment and scores for Positive, Negative, Neutral, Mixed  

https://docs.aws.amazon.com/comprehend/latest/dg/how-sentiment.html  

### Customer Reviews for Major Appliances

amazon reviews pds is no longer accessible  
!aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Major_Appliances_v1_00.tsv.gz .

**Please utilize the file included in the course Git repository: data\customer_reviews_with_sentiment.parquet.**  

**It contains nearly 97,000 customer reviews pertaining to major appliances.**

In [ ]:
## Install pyarrow package

!pip install pyarrow

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import pyarrow.parquet as pq

### Prepare Training and Test data 

In [2]:
parquet_file_name = r".\data\customer_reviews_with_sentiment.parquet"

In [3]:
df = pd.read_parquet(parquet_file_name)

In [4]:
print('Rows: {0}, Columns: {1}'.format(df.shape[0],df.shape[1]))

Rows: 96834, Columns: 16


In [5]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,sentiment
0,US,16199106,R203HPW78Z7N4K,B0067WNSZY,633038551,"FGGF3032MW Gallery Series 30"" Wide Freestandin...",Major Appliances,5,0,0,N,Y,"If you need a new stove, this is a winner.",What a great stove. What a wonderful replacem...,2015-08-31,POSITIVE
1,US,16374060,R2EAIGVLEALSP3,B002QSXK60,811766671,Best Hand Clothes Wringer,Major Appliances,5,1,1,N,Y,Five Stars,worked great,2015-08-31,POSITIVE
2,US,15322085,R1K1CD73HHLILA,B00EC452R6,345562728,Supco SET184 Thermal Cutoff Kit,Major Appliances,5,0,0,N,Y,Fast Shipping,Part exactly what I needed. Saved by purchasi...,2015-08-31,POSITIVE
3,US,32004835,R2KZBMOFRMYOPO,B00MVVIF2G,563052763,Midea WHS-160RB1 Compact Single Reversible Doo...,Major Appliances,5,1,1,N,Y,Five Stars,Love my refrigerator! ! Keeps everything cold...,2015-08-31,POSITIVE
4,US,25414497,R6BIZOZY6UD01,B00IY7BNUW,874236579,Avalon Bay Portable Ice Maker,Major Appliances,5,0,0,N,Y,Five Stars,No more running to the store for ice! Works p...,2015-08-31,POSITIVE


In [6]:
df.columns

Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date', 'sentiment'],
      dtype='object')

In [7]:
# The sentiment for the reviews is already incorporated in this file.
# In this exercise, we are using Firehose integration to assess the sentiment. So, let's drop
# sentiment column

df.drop(columns='sentiment',inplace=True)

In [9]:
df.columns

Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date'],
      dtype='object')

In [10]:
# Replace embedded new lines, tabs and carriage return
pattern = r'[\n\t\r]+'

### Submit review to Firehose Stream

In [ ]:
import boto3

In [ ]:
session = boto3.Session(region_name='us-east-1')

In [ ]:
client_firehose = session.client('firehose')

In [ ]:
kinesis_delivery_stream_name = 'CustomerReviewStream'

### Warning: Sending all 100,000 reviews would incur a cost of USD 65 for sentiment analysis.
### In this lab, we need to send only the first 10 reviews


firehose to s3 json  
https://stackoverflow.com/questions/34468319/reading-the-data-written-to-s3-by-amazon-kinesis-firehose-stream/49417680#49417680

In [ ]:
# Push Reviews to Firehose

for i in range(10):
    # Strip out any new line, tab and carriage return from json payload
    # Add a new line at the end to ensure firehose places each json record in a separate
    # row. without the new line, firehose simply places all records in a single line in S3.
    payload = re.sub(pattern,' ', df.iloc[i].to_json()) + "\n"

    print(payload)
    response = client_firehose.put_record(
        DeliveryStreamName=kinesis_delivery_stream_name,
        Record={
            'Data': payload
        }
    )

    print ('Response',response['ResponseMetadata']['HTTPStatusCode'])
    print()
'''if response['ResponseMetadata']['HTTPStatusCode'] != 200:
        print (response)
    else:
        print('.',end=' ')
'''        

### Verify Delivered Data in the S3 Bucket

If data is not visible after 15 minutes, please inspect the CloudWatch Log for the Lambda Function for any potential errors.